# Spotify Natural Language Processing Project




### By: Daniel Ford, Glady Barrios, Kevin Smith

----

# Project Goal

- The goal of this project is to use natural language processing and classification models to identify terms for predicting a readme's primary language on Github.

---

# Inital Questions 

 - What are the top 5 programming languages when searching for 'Spotify' repos on github?
 - What are the most common words we would see when searching for spotify README's
 - From these top 5 programing languages what are the most common words from these languages 
 - What are some common bigrams in the languages when using these bigrams


### Mini-data dictionary

---
| Attribute | Definition | Data Type |
| ----- | ----- | ----- |
|Repo |The username of the REPO|object |
|readme_contents |What is inside the readme | object|
|language|the programming language |object |
|lemmatized|prepared data |object |


### Important Libraries

In [12]:
import pandas as pd
import numpy as np
import unicodedata
import re, os
import json
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.stats import zscore


import time
import random
from requests import get
from bs4 import BeautifulSoup
#the aquire takes a while
import acquire

import prepare

import warnings
warnings.filterwarnings("ignore")

# Acquire
---

To acquire the data first we needed to scrape the links for the individual repos from github by iterating through the pages of search results and grabbing the 10 links from each page. 

Once this step was completed we were able to populate a dataframe with the name of the repo, the main coding language, and the contents of the README file associated with the repo by utilizing different functions that gathered those pieces of data from the github API. 

Both process were wrapped in a TQDM progress bar because the acquisition took about 45 minutes altogether, and having a progress bar allowed us to know if the function was still working, or if it had timed out.

In [2]:
df = acquire.github_df()

In [3]:
df.head()

,repo,language,readme_contents
0,zmb3/spotify,Go,\nSpotify\n=======\n\n[![GoDoc](https://godoc....
1,spotDL/spotify-downloader,Python,<!--- mdformat-toc start --slug=github --->\n\...
2,Spotifyd/spotifyd,Rust,# Spotifyd <!-- omit in toc -->\n<!-- ALL-CONT...
3,Rigellute/spotify-tui,Rust,# Spotify TUI\n\n![Continuous Integration](htt...
4,JohnnyCrazy/SpotifyAPI-NET,C#,"\n<h1 align=""center"">\n <p align=""center"">Spo..."


# Prepare 

In [4]:
df = df.dropna()

In [11]:
df= prepare_df(df,'readme_contents', extra_words = ['also', '&#9', 'e', 'f', 'ou', 'et', 'n', '1', "'", ';', '3', 'e', 'p'])

In [9]:
df.language.value_counts()

other         338
JavaScript    271
Python        187
TypeScript     71
Shell          36
C#             28
Name: language, dtype: int64

### Split the data

# Exploration 

# Modeling

# Conclusion 